In [28]:
using Rx
using BenchmarkTools

In [29]:
function withRegularArray()
    values = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in values
         push!(keep, value ^ 2)
    end
    return keep
end

withRegularArray (generic function with 1 method)

In [30]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [31]:
@CreateMapOperator(SquaredInt, Int, Int, (d) -> d ^ 2)

function withPregeneratedOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> SquaredIntMapOperator(), keep)
    return keep.values
end

withPregeneratedOperator (generic function with 1 method)

In [32]:
function withBaseMapOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> map(Int, Int, (d) -> d ^ 2), keep)
    return keep.values
end

withBaseMapOperator (generic function with 1 method)

In [33]:
@btime withRegularArray();

  7.164 μs (11 allocations: 24.33 KiB)


In [34]:
@btime withPregeneratedOperator();

  7.373 μs (15 allocations: 24.41 KiB)


In [35]:
@btime withBaseMapOperator();

  48.144 μs (1483 allocations: 47.36 KiB)
